In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [29]:
dataset1 = pd.read_csv("train.csv")
dataset2 = pd.read_csv("test.csv")


In [30]:
dataset1 = dataset1.drop(['PassengerId'], axis=1)
dataset1 = dataset1.drop(['Cabin'], axis=1)
dataset1 = dataset1.drop(['Name'], axis=1)

In [31]:
dataset2 = dataset2.drop(['PassengerId'], axis=1)
dataset2 = dataset2.drop(['Cabin'], axis=1)
dataset2 = dataset2.drop(['Name'], axis=1)

In [32]:
x = dataset1.iloc[ : , :-1 ].values
y = dataset1.iloc[ : ,-1 ].values

In [33]:
x2 = dataset2.iloc[ : , : ].values

In [34]:
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8476 non-null   object 
 2   Destination   8511 non-null   object 
 3   Age           8514 non-null   float64
 4   VIP           8490 non-null   object 
 5   RoomService   8512 non-null   float64
 6   FoodCourt     8510 non-null   float64
 7   ShoppingMall  8485 non-null   float64
 8   Spa           8510 non-null   float64
 9   VRDeck        8505 non-null   float64
 10  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(4)
memory usage: 687.8+ KB


In [35]:
dataset2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    4190 non-null   object 
 1   CryoSleep     4184 non-null   object 
 2   Destination   4185 non-null   object 
 3   Age           4186 non-null   float64
 4   VIP           4184 non-null   object 
 5   RoomService   4195 non-null   float64
 6   FoodCourt     4171 non-null   float64
 7   ShoppingMall  4179 non-null   float64
 8   Spa           4176 non-null   float64
 9   VRDeck        4197 non-null   float64
dtypes: float64(6), object(4)
memory usage: 334.3+ KB


In [36]:
print(x2[ : , 0:4 ])

[['Earth' True 'TRAPPIST-1e' 27.0]
 ['Earth' False 'TRAPPIST-1e' 19.0]
 ['Europa' True '55 Cancri e' 31.0]
 ...
 ['Mars' True '55 Cancri e' nan]
 ['Europa' False nan nan]
 ['Earth' True 'PSO J318.5-22' 43.0]]


In [37]:
print(y)

[False  True False ...  True False  True]


**AGE**

In [38]:
from sklearn.impute import SimpleImputer 
imputer = SimpleImputer(missing_values=np.nan, strategy='mean' )
imputer.fit(x[ : , 3:4 ])
x[ : , 3:4 ] = imputer.transform(x[ : , 3:4 ])

imputer.fit(x2[ : , 3:4 ])
x2[ : , 3:4 ] = imputer.transform(x2[ : , 3:4 ])

**Room service to VRdeck**

In [39]:
imputer2 = SimpleImputer(missing_values=np.nan, strategy='most_frequent' )
imputer2.fit(x[ : , 5:10 ])
x[ : , 5:10 ] = imputer2.transform(x[ : , 5:10 ])

imputer2.fit(x2[ : , 5:10 ])
x2[ : , 5:10 ] = imputer2.transform(x2[ : , 5:10 ])

**encoding Home planet**

In [40]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct=ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0] )] , remainder='passthrough' )

x = np.array(ct.fit_transform(x))

x2= np.array(ct.fit_transform(x2))

**encoding Cyrosleep**

In [41]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[: , 4]= le.fit_transform(x[: , 4])
x2[: , 4]= le.fit_transform(x2[: , 4])

**encoding destination**

In [42]:
ct2=ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [5] )] , remainder='passthrough' )
x = np.array(ct2.fit_transform(x))

x2 = np.array(ct2.fit_transform(x2))

**encoding VIP**

In [43]:
le2 = LabelEncoder()
x[: , 10]= le2.fit_transform(x[: , 10])
x2[: , 10]= le2.fit_transform(x2[: , 10])

**encoding dependent variable of dataset1 = y**



In [44]:
le3 = LabelEncoder()
y= le3.fit_transform(y)

**missing values**

In [45]:
imputer3 = SimpleImputer(missing_values=np.nan, strategy='most_frequent' )
imputer3.fit(x[ : , : ])
x[ : , : ] = imputer3.transform(x[ : , : ])

imputer3.fit(x2[ : , : ])
x2[ : , : ] = imputer3.transform(x2[ : , : ])

**MODEL**

In [46]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(x, y)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [47]:
y_pred = classifier.predict((x2))


In [48]:
print(y_pred)

[1 0 1 ... 1 1 0]


In [49]:
y_pred = y_pred.astype(str)
y_pred = np.char.replace(y_pred, '0', 'False')
y_pred = np.char.replace(y_pred, '1', 'True')
print(y_pred)

['True' 'False' 'True' ... 'True' 'True' 'False']


**SUBMISSION!!**

In [50]:
submission = pd.read_csv("sample_submission.csv")



In [51]:
submission['Transported'] = y_pred

In [52]:
submission.to_csv('submission.csv', index = False)


In [53]:
my_submission = pd.read_csv("submission.csv")

In [54]:
my_submission

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True
